## 🤖 Agentic AI with OpenAI Function Calling + Pydantic

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import json, os
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


# ✅ Step 1: Define function input models using pydantic
class ReminderRequest(BaseModel):
    task: str = Field(..., description="What to be reminded about")
    time: str = Field(..., description="When to remind (e.g., 2025-05-27 20:00)")

# ✅ Step 2: Define your fake reminder function
def set_reminder(task: str, time: str):
    return f"Reminder set: '{task}' at {time}"

# ✅ Step 3: Initialize OpenAI client
client = OpenAI(api_key = api_key)  # Uses OPENAI_API_KEY from environment

# ✅ Step 4: Define the tool (function) for GPT
tools = [
    {
        "type": "function",
        "function": {
            "name": "set_reminder",
            "description": "Set a task reminder",
            "parameters": ReminderRequest.schema()
        }
    }
]

# ✅ Step 5: Simulate a user message
user_message = "Remind me to join the team meeting at 4 PM today"

# ✅ Step 6: Send to GPT
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": user_message}],
    tools=tools,
    tool_choice="auto"
)

# ✅ Step 7: Handle tool call (if any)
tool_calls = response.choices[0].message.tool_calls

if tool_calls:
    for call in tool_calls:
        func_name = call.function.name
        args = json.loads(call.function.arguments)

        if func_name == "set_reminder":
            result = set_reminder(**args)
            print("🔔", result)
else:
    print("🤖", response.choices[0].message.content)


C:\Users\aminu\AppData\Local\Temp\ipykernel_10512\568098759.py:33: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  "parameters": ReminderRequest.schema()


🔔 Reminder set: 'Join the team meeting' at 2023-11-02 16:00


# R&D

## 🤖 Agentic AI with OpenAI Function Calling + Pydantic
 
### This notebook shows how to use **Pydantic** and **OpenAI's Function Calling** to create a simple AI agent that:
#### - Understands user intent
#### - Chooses the right function (get weather or set a reminder)
#### - Validates the parameters before acting

### 🔧 Requirements:

In [7]:
!pip install openai pydantic 

In [15]:
from pydantic import BaseModel, Field
from typing import Literal
import json
from dotenv import load_dotenv
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key = api_key)

load_dotenv()


True

In [16]:
# ✅ Step 1: Define Function Schemas using Pydantic

class WeatherRequest(BaseModel):
    city: str = Field(..., description="City to get weather for")

class ReminderRequest(BaseModel):
    task: str = Field(..., description="What to be reminded about")
    time: str = Field(..., description="When to remind (e.g., 2025-05-27 20:00)")

# ✅ Purpose:
# This model defines the data structure for a weather query request.

# ✅ Fields:
# city: A required string field (due to ...) with a description to indicate that it's the name of the city for which the weather is requested.

In [17]:
# ✅ Step 2: Define Available Functions for OpenAI

functions = [
    {
        "name": "get_weather",
        "description": "Get weather for a city",
        "parameters": WeatherRequest.model_json_schema()
    },
    {
        "name": "set_reminder",
        "description": "Set a task reminder",
        "parameters": ReminderRequest.model_json_schema()
    }
]

In [18]:
# ✅ Step 3: Simulate User Input

user_message = "Remind me to call mom at 8 PM tonight"

In [21]:
# ✅ Step 4: Send Message to OpenAI with Function Calling

response = client.chat.completions.create(
    model="gpt-4o",  # Or another model like gpt-4o or gpt-4-turbo
    messages=[
        {"role": "user", "content": user_message}
    ],
    functions=functions,
    function_call="auto"
)

In [30]:
response

ChatCompletion(id='chatcmpl-Bbq3XXoT1a0X6Ul3fxKRuqmxPz41T', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"task":"call mom","time":"2023-10-06 20:00:00"}', name='set_reminder'), tool_calls=None))], created=1748357939, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_9bddfca6e2', usage=CompletionUsage(completion_tokens=31, prompt_tokens=128, total_tokens=159, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [32]:
function_call = response.choices[0].message.function_call
print("Response function_call:", function_call)

Response function_call: FunctionCall(arguments='{"task":"call mom","time":"2023-10-06 20:00:00"}', name='set_reminder')


In [36]:
# Get the function name
name = function_call.name

# Parse the arguments JSON string to a Python dict
args = json.loads(function_call.arguments)

# Get task and time
task = args["task"]
time = args["time"]

print("Function name:", name)
print("Task:", task)
print("Time:", time)

Function name: set_reminder
Task: call mom
Time: 2023-10-06 20:00:00


'{"task":"call mom","time":"2023-10-06 20:00:00"}'

In [41]:
# ✅ Step 5: Parse Function Call from Response

func_call = response.choices[0].message.function_call

if func_call:
    print(f"🔍 Function to call: {func_call.name}")
    print(f"🧾 Arguments: {func_call.arguments}")

    args_dict = json.loads(function_call.arguments)

    if func_call.name == "set_reminder":
        try:
            args = ReminderRequest.model_validate(args_dict)
            print(f"✅ Reminder Task: {args.task}")
            print(f"⏰ Reminder Time: {args.time}")
            # Here you can invoke your own reminder setting logic
        except Exception as e:
            print(f"❌ Invalid reminder arguments: {e}")

    elif func_call.name == "get_weather":
        try:
            args = WeatherRequest.model_validate(args_dict)
            print(f"🌤️ Fetching weather for: {args.city}")
            # Here you can invoke your own weather-fetching logic
        except Exception as e:
            print(f"❌ Invalid weather arguments: {e}")
else:
    print("🤖 No function call was made.")

🔍 Function to call: set_reminder
🧾 Arguments: {"task":"call mom","time":"2023-10-06 20:00:00"}
✅ Reminder Task: call mom
⏰ Reminder Time: 2023-10-06 20:00:00


In [43]:
print(ReminderRequest.model_json_schema())

{'properties': {'task': {'description': 'What to be reminded about', 'title': 'Task', 'type': 'string'}, 'time': {'description': 'When to remind (e.g., 2025-05-27 20:00)', 'title': 'Time', 'type': 'string'}}, 'required': ['task', 'time'], 'title': 'ReminderRequest', 'type': 'object'}
